Group 5: LI LINGYU, LIU YICHAO, WU JINGYAN, YANG QINGSHAN, YE FANGDA

In [14]:
# Construct time features

chicago_clean['Date'] = pd.to_datetime(chicago_clean['Date'], format='%m/%d/%Y %I:%M:%S %p')
chicago_clean['Hour'] = chicago_clean['Date'].dt.hour
chicago_clean['Day'] = chicago_clean['Date'].dt.day
chicago_clean['Month'] = chicago_clean['Date'].dt.month
chicago_clean['DayOfWeek'] = chicago_clean['Date'].dt.dayofweek
chicago_clean['Year'] = chicago_clean['Date'].dt.year
chicago_clean['IsWeekend'] = chicago_clean['DayOfWeek'].isin([5,6]).astype(int)
chicago_clean['TimeOfDay'] = pd.cut(chicago_clean['Hour'], 
                                   bins=[0,6,12,18,24], 
                                   labels=['Night','Morning','Afternoon','Evening'])
chicago_clean['Season'] = pd.cut(chicago_clean['Month'],
                                bins=[-1,2,5,8,11],
                                labels=['Winter','Spring','Summer','Fall'])

# Drop initial date column
chicago_clean = chicago_clean.drop('Date', axis=1)


['../data/chicago_clean_final_arrest.csv', '../data/chicago_clean_final_not_arrest.csv']


In [ ]:
#construct spatial features
chicago_clean[['grid_x', 'grid_y']] = chicago_clean['grid'].str.split('_', expand=True).astype(int)
chicago_clean['is_border'] = (
    (chicago_clean['grid_x'].isin([0, 10])) | 
    (chicago_clean['grid_y'].isin([0, 10]))
).astype(int)


In [ ]:
#construct environmental features

top_locations = chicago['Location Description'].value_counts().head(5).index.tolist()
for loc in top_locations:
    chicago_clean[f'Location_{loc}'] = (chicago['Location Description'] == loc).astype(int)

chicago_clean['Community_Area'] = chicago['Community Area']

chicago_clean['District'] = chicago['District']
chicago_clean['Ward'] = chicago['Ward']

fbi_risk_mapping = {
    '04': 'High',   
    '08': 'Medium', 
    '06': 'Low'     
}
chicago_clean['FBI_Risk'] = chicago['FBI Code'].map(fbi_risk_mapping)
fbi_risk_dummies = pd.get_dummies(chicago_clean['FBI_Risk'], prefix='FBI_Risk')
chicago_clean = pd.concat([chicago_clean, fbi_risk_dummies], axis=1)
chicago_clean = chicago_clean.drop('FBI_Risk', axis=1)


In [ ]:
chicago_clean = pd.get_dummies(chicago_clean, columns=["Season", "TimeOfDay", "DayOfWeek"])
chicago_clean